# Priyanka Vinod Thesis

#### Load all packages and functions 

In [1]:
## import all packages
import time
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from scipy.stats import mode
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam 
from keras.losses import binary_crossentropy
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, LSTM, GRU 
from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import auc, f1_score, roc_curve, accuracy_score, roc_auc_score, balanced_accuracy_score, precision_recall_curve, confusion_matrix, classification_report 

In [2]:
## load OpenFace data and return a dataframe 
def load_OF_data(path):
    df = pd.read_csv(path, index_col = 0) 
    print('Dimensions of OpenFace data from R:', df.shape)
    print('Column names of OpenFace data from R:\n', list(df))
    return df   

In [3]:
## load FaceReader data and return a dataframe 
def load_FR_data(path):
    df = pd.read_csv(path, index_col = 0) 
    print('\nDimensions of FaceReader data from R:', df.shape)
    print('Column names of FaceReader data from R:\n', list(df))
    return df   

In [4]:
## load survey data and return a dataframe
def load_survey_data(path):
    df = pd.read_csv(path, delimiter = ';') 
    print('\nSurvey data: \n', df)
    return df

In [5]:
## process the survey data
def survey_process(df):
    a = df['index']
    b = df['pinv']
    return a, b 

In [6]:
## split the entire df into cumulative (features+labels) train and test parts by index-matching
def get_rows(trn_ind, tst_ind, of_data):
    df_trn = of_data[of_data['index'].isin(trn_ind)]
    df_tst = of_data[of_data['index'].isin(tst_ind)]  
    return df_trn, df_tst 

In [7]:
## split the cumulative train and test parts into 4 train and test parts (features and labels respectively) 
def get_train_test_dfs(df_trn, df_tst):
    y_train = df_trn['y']
    y_test = df_tst['y']
    cols = ['y', 'index']
    X_train = df_trn.drop(cols, axis = 1)   
    X_test = df_tst.drop(cols, axis = 1)   
    return X_train, X_test, y_train, y_test   

In [8]:
## remove used indices to ensure indices do not overlap
def new_ind_pinv(index, pinv, trn):
    df = pd.concat([index, pinv], axis = 1)
    data = df[df['index'].isin(trn.tolist())]
    return data['index'], data['pinv']  

In [9]:
## setting up the parameter grids for HP tuning
def parameter_grid(timestep, features):
    grid = [{'units': [32, 64, 128, 256],                   
             'dropout': [0.2, 0.3, 0.4, 0.5],                 
             'epochs': [50, 75, 100],                     
             'batch_size': [16, 32, 64],   
             'rate': [0.01, 0.001, 0.0001],
             'timestep': [timestep],
             'features': [features]}]    
    return grid

In [10]:
# setting up multiple GridSearchCV objects, 1 for each algorithm (we have 2 for LSTM and GRU)
def grid_search_objects(grid):
    gridcvs = {}
    inner_cv = RepeatedStratifiedKFold(n_splits = 2, n_repeats = 2, random_state = None) 

    model1 = KerasClassifier(build_fn = base_GRU) 
    model2 = KerasClassifier(build_fn = base_LSTM) 
             
    for pgrid, est, name in zip((grid, grid), 
                                (model1, model2), 
                                ('GRU', 'LSTM')):  
        gcv = GridSearchCV(estimator = est,
                           param_grid = pgrid,
                           scoring = 'accuracy',
                           n_jobs = -1,
                           cv = inner_cv,
                           verbose = 0,
                           refit = True, 
                           return_train_score = True)
        gridcvs[name] = gcv
    return gridcvs, inner_cv   

In [11]:
## scale train and test data to range [0, 1] 
def scale_data(train, test): 
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train.values)
    test_scaled = scaler.transform(test.values)
    return train_scaled, test_scaled

In [12]:
## find mode for reshaped labels
def find_mode(data):
    m = mode(data, axis = 1)
    return m[0]

In [13]:
## reshape data
def reshape_data(X_train, X_test, y_train, y_test, timestep):
    sample1 = int(X_train.shape[0]/timestep)
    sample2 = int(X_test.shape[0]/timestep)
    features = X_train.shape[1]
    X_train = X_train.reshape(sample1, timestep, features)
    X_test = X_test.reshape(sample2, timestep, features)
    y_train = y_train.values.reshape(sample1, timestep)
    y_train = find_mode(y_train)
    y_test = y_test.values.reshape(sample2, timestep)
    y_test = find_mode(y_test)   
    return X_train, X_test, y_train, y_test

In [14]:
## define GRU model 
def base_GRU(units = 32, dropout = 0.2, rate = 0.001, timestep = 100, features = 16):
    model =  Sequential()
    model.add(GRU(units, input_shape = (timestep, features)))  
    model.add(Dropout(dropout))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    # compile model
    opt = Adam(learning_rate = rate)
    model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [15]:
## define LSTM model 
def base_LSTM(units = 32, dropout = 0.2, rate = 0.001, timestep = 100, features = 16):
    model =  Sequential()
    model.add(LSTM(units, input_shape = (timestep, features))) 
    model.add(Dropout(dropout))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    # compile model
    opt = Adam(lr = rate)
    model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [16]:
def scores(y_true, y_pred):  
    acc = accuracy_score(y_true, y_pred)
    bacc = balanced_accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    target_names = ['No', 'Yes']
    cr = classification_report(y_true, y_pred, target_names = target_names)
    roc = roc_auc_score(y_true, y_pred)
    return acc, bacc, f1, cm, cr, roc

In [17]:
def plot_cm(cm):
    plt.figure(figsize = (5, 5))
    ax = plt.subplot()
    sns.heatmap(cm, annot = True, fmt = ".0f", linewidths = .5, square = True, cmap = 'Blues_r', ax = ax);
    ax.set_ylabel('True labels'); 
    ax.set_xlabel('Predicted labels');
    ax.xaxis.set_ticklabels(['No', 'Yes']); 
    ax.yaxis.set_ticklabels(['No', 'Yes']);
    plt.title('Confusion Matrix ', size = 15);
    plt.show()

In [18]:
def roc_plot(lr_probs, y_true, name):
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(y_true))]
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(y_true, ns_probs)
    lr_auc = roc_auc_score(y_true, lr_probs)
    # summarize scores
    print('        ROC AUC = %.3f' % (lr_auc))
    ns_fpr, ns_tpr, _ = roc_curve(y_true, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_true, lr_probs)
    plt.plot(ns_fpr, ns_tpr, linestyle = '--', label = 'No Skill')
    plt.plot(lr_fpr, lr_tpr, marker = '.', label = name)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

In [19]:
def prc_plot(lr_probs, y_true, y_pred, name):
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(y_true))]
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]
    # calculate scores
    lr_precision, lr_recall, _ = precision_recall_curve(y_true, lr_probs)
    lr_f1, lr_auc = f1_score(y_true, y_pred), auc(lr_recall, lr_precision)
    # summarize scores
    print('        F1-score = %.3f, PRC AUC = %.3f' % (lr_f1, lr_auc))
    # plot the precision-recall curves
    no_skill = len(y_true[y_true == 1]) / len(y_true)
    plt.plot([0, 1], [no_skill, no_skill], linestyle = '--', label = 'No Skill')
    plt.plot(lr_recall, lr_precision, marker = '.', label = name)
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PRC Curve')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

In [20]:
def roc_combined(tprs, aucs, mean_fpr, i):
    plt.plot([0, 1], [0, 1], linestyle='--', label='No skill')
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc))
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Cross-Validation ROC')
    plt.legend()
    plt.show()

In [21]:
## implement stratified k-fold nested cross validation 
def nested_stratifiedKfold(index, pinv, data, gridcvs, inner_cv, timestep):
    for name, gs_est in sorted(gridcvs.items()):
        print('\nAlgorithm:', name, '\n')
        print('    Details for each fold of outer loop:\n')
    
        outer_ascores_segment = []
        outer_ascores_video = []
        outer_fscores_segment = []
        outer_fscores_video = []
        outer_cm_segment = []
        outer_cm_video = []
        outer_roc_segment = []
        outer_roc_video = []
        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        i = 0
        fold = 1
        gridlst = []
                
        outer_cv = RepeatedStratifiedKFold(n_splits = 4, n_repeats = 1, random_state = None)  #shuffle = True,  
                
        for train_index, test_index in outer_cv.split(index, pinv):
            print('        Fold', fold, '/ 4')
            i_trn, i_tst = index.iloc[train_index], index.iloc[test_index]
            p_trn, p_tst = pinv.iloc[train_index], pinv.iloc[test_index]
            print('        Startup indices used for training:', i_trn.values, '\n'
                  '        Startup indices used for testing:', i_tst.values)
                       
            df_trn, df_tst = get_rows(i_trn, i_tst, data)
            xtrain, xtest, ytrain, ytest = get_train_test_dfs(df_trn, df_tst)
            XX_train, XX_test = scale_data(xtrain, xtest)        
            
            ## just to verify indices used for HP tuning; ignore this
            print('        Startup indices used for hyperparameter tuning:')
            index_new, pinv_new = new_ind_pinv(index, pinv, i_trn.values)
            for train_index, test_index in inner_cv.split(index_new, pinv_new):
                i_trn, i_tst = index_new.iloc[train_index], index_new.iloc[test_index]
                print('          Training:', i_trn.values, '\n'
                      '          Validation:', i_tst.values)
            ## end of code     
                
            X_train, X_test, y_train, y_test = reshape_data(XX_train, XX_test, ytrain, ytest, timestep)            
            print('\n        Dimensions of train and test data (features):', X_train.shape, ',', X_test.shape)
            print('        Dimensions of train and test data (targets):', y_train.shape, ',', y_test.shape, '\n')
            
            ## fitting the model
            time_start = time.time()
            gridcvs[name].fit(X_train, y_train, verbose = 1)  
            
            print("        Time taken for training:", str(round(time.time() - time_start, 2)) + "s")
            print('        Best hyperparameters found for current fold:\n', '        ', gridcvs[name].best_params_, '\n')
            y_true = y_test
            y_pred = gridcvs[name].predict(X_test)
            lr_probs = gridcvs[name].predict_proba(X_test)            
                
            ## plot roc and prc curves
            roc_plot(lr_probs, y_true, name)
            prc_plot(lr_probs, y_true, y_pred, name)
            # for combined roc curve
            fpr, tpr, _ = roc_curve(y_true, lr_probs[:, 1])
            tprs.append(np.interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
                                       
            ## calculating scores
            acc, bacc, fscore, cm, cr, roc = scores(y_true, y_pred) #, lr_probs)
            print('        Segment-level test accuracy: %.2f%%' % (100 * acc)) 
            outer_ascores_segment.append(acc) 
            print('        Segment-level balanced accuracy: %.2f%%' % (100 * bacc))
            print('        Segment-level F1-score: %.2f%%' % (100 * fscore))    
            outer_fscores_segment.append(fscore) 
            print('        Segment-level ROC-AUC score: %.2f%%' % (100 * roc))    
            outer_roc_segment.append(roc) 
            print('        Confusion matrix:\n')
            outer_cm_segment.append(cm)
            plot_cm(cm)
            print('        Classification report:\n', cr)
            
            count_test = 6                     # count of test videos                      
            s = int(y_test.shape[0]/count_test)                  
            p = mode(y_true.reshape(count_test, s), axis = 1)
            q = mode(y_pred.reshape(count_test, s), axis = 1) 
            acc, bacc, fscore, cm, cr, roc = scores(p[0], q[0])   
            print('        Video-level test accuracy: %.2f%%' % (100 * acc))
            outer_ascores_video.append(acc)
            print('        Video-level balanced accuracy: %.2f%%' % (100 * bacc))
            print('        Video-level F1-score: %.2f%%' % (100 * fscore))
            outer_fscores_video.append(fscore)
            print('        Video-level ROC-AUC score: %.2f%%' % (100 * roc))
            outer_roc_video.append(roc)
            print('        Confusion matrix:\n')
            outer_cm_video.append(cm)
            plot_cm(cm)
            print('        Classification report:\n', cr) 
            print('       ', 100 * '*', '\n')
            
            fold += 1
            i += 1
                
        print('\n    Average accuracy over all folds:')
        print('        Segment-level accuracy %.2f%% (%.2f)' % (np.mean(outer_ascores_segment) * 100, 
                                                       np.std(outer_ascores_segment) * 100)) 
        print('        Video-level accuracy %.2f%% (%.2f)' % (np.mean(outer_ascores_video) * 100, 
                                                     np.std(outer_ascores_video) * 100))  
        print('\n    Average F1-score over all folds:')
        print('        Segment-level F1-score %.2f%% (%.2f)' % (np.mean(outer_fscores_segment) * 100, 
                                                       np.std(outer_fscores_segment) * 100)) 
        print('        Video-level F1-score %.2f%% (%.2f)' % (np.mean(outer_fscores_video) * 100, 
                                                     np.std(outer_fscores_video) * 100)) 
        print('\n    Average ROC-AUC score over all folds:')
        print('        Segment-level ROC-AUC score %.2f%% (%.2f)' % (np.mean(outer_roc_segment) * 100, 
                                                       np.std(outer_roc_segment) * 100)) 
        print('        Video-level ROC-AUC score %.2f%% (%.2f)' % (np.mean(outer_roc_video) * 100, 
                                                     np.std(outer_roc_video) * 100)) 
        print('\n    Confusion matrix over all folds:')
        print('        Segment-level CM: \n')        
        avg_cm = np.mean(outer_cm_segment, axis = 0).astype(np.int16)
        sum_cm = np.sum(outer_cm_segment, axis = 0).astype(np.int16)
        plot_cm(avg_cm)
        plot_cm(sum_cm)
        print('        Video-level CM: \n')
        avg_cm = np.mean(outer_cm_video, axis = 0).astype(np.int16)
        sum_cm = np.sum(outer_cm_video, axis = 0).astype(np.int16)
        plot_cm(avg_cm)  
        plot_cm(sum_cm)
        roc_combined(tprs, aucs, mean_fpr, i)

In [22]:
def experiment1(timestep): 
    # load OpenFace data from R 
    path1 = 'data/expt1_of.csv'
    of_data = load_OF_data(path1)
    
    # load FaceReader data from R 
    path2 = 'data/expt1_fr.csv'
    fr_data = load_FR_data(path2)
    
    # load survey data
    path3 = 'data/startups.csv'
    survey_data = load_survey_data(path3)
    
    # number of features for models
    features = of_data.shape[1] - 2
    
    # access index and pinv columns from survey data 
    index, pinv = survey_process(survey_data)
    
    # get parameter grid for HP tuning 
    grid = parameter_grid(timestep, features)
    
    # initialise GridSearchCV object
    gridcvs, inner_cv = grid_search_objects(grid)
      
    # perform nested k-fold cross validation for OpenFace data
    print('\n', 50 * '-', ' FOR OPENFACE DATA ', 50 * '-')
    nested_stratifiedKfold(index, pinv, of_data, gridcvs, inner_cv, timestep)
        
    # perform nested k-fold cross validation for FaceReader data
    print('\n', 50 * '-', ' FOR FACEREADER DATA ', 50 * '-')
    nested_stratifiedKfold(index, pinv, fr_data, gridcvs, inner_cv, timestep)  

In [ ]:
def experiment2(timestep): 
    # load OpenFace data from R 
    path1 = 'data/expt2_of.csv'
    of_data = load_OF_data(path1)
    
    # load FaceReader data from R 
    path2 = 'data/expt2_fr.csv'
    fr_data = load_FR_data(path2)
    
    # load survey data
    path3 = 'data/startups.csv'
    survey_data = load_survey_data(path3)
    
    # number of features for models
    features = of_data.shape[1] - 2
    
    # access index and pinv columns from survey data 
    index, pinv = survey_process(survey_data)
    
    # get parameter grid for HP tuning 
    grid = parameter_grid(timestep, features)
    
    # initialise GridSearchCV object
    gridcvs, inner_cv = grid_search_objects(grid)
      
    # perform nested k-fold cross validation for OpenFace data
    print('\n', 50 * '-', ' FOR OPENFACE DATA ', 50 * '-')
    nested_stratifiedKfold(index, pinv, of_data, gridcvs, inner_cv, timestep)
        
    # perform nested k-fold cross validation for FaceReader data
    print('\n', 50 * '-', ' FOR FACEREADER DATA ', 50 * '-')
    nested_stratifiedKfold(index, pinv, fr_data, gridcvs, inner_cv, timestep)  

In [ ]:
def experiment3(timestep): 
    # load OpenFace data from R 
    path1 = 'data/expt3_of.csv'
    of_data = load_OF_data(path1)
    
    # load FaceReader data from R 
    path2 = 'data/expt3_fr.csv'
    fr_data = load_FR_data(path2)
    
    # load survey data
    path3 = 'data/startups.csv'
    survey_data = load_survey_data(path3)
    
    # number of features for models
    features = of_data.shape[1] - 2
    
    # access index and pinv columns from survey data 
    index, pinv = survey_process(survey_data)
    
    # get parameter grid for HP tuning 
    grid = parameter_grid(timestep, features)
    
    # initialise GridSearchCV object
    gridcvs, inner_cv = grid_search_objects(grid)
      
    # perform nested k-fold cross validation for OpenFace data
    print('\n', 50 * '-', ' FOR OPENFACE DATA ', 50 * '-')
    nested_stratifiedKfold(index, pinv, of_data, gridcvs, inner_cv, timestep)
        
    # perform nested k-fold cross validation for FaceReader data
    print('\n', 50 * '-', ' FOR FACEREADER DATA ', 50 * '-')
    nested_stratifiedKfold(index, pinv, fr_data, gridcvs, inner_cv, timestep)  

In [ ]:
def experiment4(timestep): 
    # load OpenFace data from R 
    path1 = 'data/expt4_of.csv'
    of_data = load_OF_data(path1)
    
    # load FaceReader data from R 
    path2 = 'data/expt4_fr.csv'
    fr_data = load_FR_data(path2)
    
    # load survey data
    path3 = 'data/startups.csv'
    survey_data = load_survey_data(path3)
    
    # number of features for models
    features = of_data.shape[1] - 2
    
    # access index and pinv columns from survey data 
    index, pinv = survey_process(survey_data)
    
    # get parameter grid for HP tuning 
    grid = parameter_grid(timestep, features)
    
    # initialise GridSearchCV object
    gridcvs, inner_cv = grid_search_objects(grid)
      
    # perform nested k-fold cross validation for OpenFace data
    print('\n', 50 * '-', ' FOR OPENFACE DATA ', 50 * '-')
    nested_stratifiedKfold(index, pinv, of_data, gridcvs, inner_cv, timestep)
        
    # perform nested k-fold cross validation for FaceReader data
    print('\n', 50 * '-', ' FOR FACEREADER DATA ', 50 * '-')
    nested_stratifiedKfold(index, pinv, fr_data, gridcvs, inner_cv, timestep)  

### Experiment 1

In [23]:
# feature Set 1
experiment1(100)

### Experiment 2

In [24]:
# feature Set 2
experiment2(100)

### Experiment 3

In [25]:
# feature Set 3
experiment3(100)

### Experiment 4

In [26]:
# feature Set 4
experiment4(100)